### Standard Lib Imports

In [1]:
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [2]:
from src import config
from src.helper import *

### Load Data

In [75]:
prices = read_data('prices.csv')
product_details = read_data('product details.csv')

In [76]:
prices.head()

,product_id,sale_a1,sale_a2,sale_a3,sale_a4,purchase_a1,purchase_a2,purchase_a3,purchase_a4,valid_from,valid_to
0,270239865,18199,16799,12999,12199,10097,7851,3576,3290,2020-01-14T11:04:45.153+01:00,2020-01-14T12:45:30.042+01:00
1,337148188,242999,206399,197999,193999,177477,147779,141648,35553,2020-06-17T09:43:29.921+02:00,2020-06-18T09:16:31.401+02:00
2,341257331,70299,65399,57899,40499,54206,47594,44206,0,2020-02-28T10:50:17.647+01:00,2020-02-28T11:36:39.077+01:00
3,330479809,4099,3799,2299,1099,1287,1084,0,0,2020-02-28T11:39:46.102+01:00,2020-03-02T12:46:29.355+01:00
4,319791009,70799,61399,57899,32299,56158,48876,44067,22051,2020-01-27T08:15:27.795+01:00,2020-01-27T14:00:10.857+01:00


In [81]:
product_details = product_details.rename({'product_id*31': 'product_id'}, axis=1)
product_details.head(3)

,product_id,manufacturer,main_category
0,283509384,Apple,Handy
1,241663414,Tamron,Kameras
2,103621251,ASUSTeKCOMPUTER,Tablets


In [82]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655674 entries, 0 to 655673
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   product_id   655674 non-null  int64 
 1   sale_a1      655674 non-null  int64 
 2   sale_a2      655674 non-null  int64 
 3   sale_a3      655674 non-null  int64 
 4   sale_a4      655674 non-null  int64 
 5   purchase_a1  655674 non-null  int64 
 6   purchase_a2  655674 non-null  int64 
 7   purchase_a3  655674 non-null  int64 
 8   purchase_a4  655674 non-null  int64 
 9   valid_from   655674 non-null  object
 10  valid_to     655674 non-null  object
dtypes: int64(9), object(2)
memory usage: 55.0+ MB


In [83]:
prices['valid_from'] = pd.to_datetime(prices['valid_from'], utc=True)

In [62]:
# There is an issue while trying to convert 'valid_to' column because of some erroneuous values

There are some rows in the data which have some invalid/missing information. Let's drop these rows. 

In [84]:
problem_idx = []

for i, ts in enumerate(prices['valid_to']):
    try:
        pd.to_datetime(ts)
    except:
        problem_idx.append(i)

In [64]:
len(problem_idx)

622

In [65]:
prices = prices.drop(problem_idx).reset_index(drop=True)

In [66]:
# Now convert 'valid_to' to datetime column
prices['valid_to'] = pd.to_datetime(prices['valid_to'], utc=True)

In [67]:
prices.head()

,product_id,sale_a1,sale_a2,sale_a3,sale_a4,purchase_a1,purchase_a2,purchase_a3,purchase_a4,valid_from,valid_to
0,270239865,18199,16799,12999,12199,10097,7851,3576,3290,2020-01-14 10:04:45.153000+00:00,2020-01-14 11:45:30.042000+00:00
1,337148188,242999,206399,197999,193999,177477,147779,141648,35553,2020-06-17 07:43:29.921000+00:00,2020-06-18 07:16:31.401000+00:00
2,341257331,70299,65399,57899,40499,54206,47594,44206,0,2020-02-28 09:50:17.647000+00:00,2020-02-28 10:36:39.077000+00:00
3,330479809,4099,3799,2299,1099,1287,1084,0,0,2020-02-28 10:39:46.102000+00:00,2020-03-02 11:46:29.355000+00:00
4,319791009,70799,61399,57899,32299,56158,48876,44067,22051,2020-01-27 07:15:27.795000+00:00,2020-01-27 13:00:10.857000+00:00


In [68]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655052 entries, 0 to 655051
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   product_id   655052 non-null  int64              
 1   sale_a1      655052 non-null  int64              
 2   sale_a2      655052 non-null  int64              
 3   sale_a3      655052 non-null  int64              
 4   sale_a4      655052 non-null  int64              
 5   purchase_a1  655052 non-null  int64              
 6   purchase_a2  655052 non-null  int64              
 7   purchase_a3  655052 non-null  int64              
 8   purchase_a4  655052 non-null  int64              
 9   valid_from   655052 non-null  datetime64[ns, UTC]
 10  valid_to     655052 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), int64(9)
memory usage: 55.0 MB


In [69]:
prices['valid_period'] = prices['valid_to'] - prices['valid_from']

In [73]:
prices['margin_a1'] = prices['sale_a1'] - prices['purchase_a1']
prices['margin_a2'] = prices['sale_a2'] - prices['purchase_a2']
prices['margin_a3'] = prices['sale_a3'] - prices['purchase_a3']
prices['margin_a4'] = prices['sale_a4'] - prices['purchase_a4']

prices['avg_margin'] = round((prices['margin_a1'] + prices['margin_a2'] + prices['margin_a3'] + prices['margin_a4'])/4, 2)

In [74]:
prices.head()

,product_id,sale_a1,sale_a2,sale_a3,sale_a4,purchase_a1,purchase_a2,purchase_a3,purchase_a4,valid_from,valid_to,valid_period,margin_a1,margin_a2,margin_a3,margin_a4,avg_margin
0,270239865,18199,16799,12999,12199,10097,7851,3576,3290,2020-01-14 10:04:45.153000+00:00,2020-01-14 11:45:30.042000+00:00,0 days 01:40:44.889000,8102,8948,9423,8909,8845.50
1,337148188,242999,206399,197999,193999,177477,147779,141648,35553,2020-06-17 07:43:29.921000+00:00,2020-06-18 07:16:31.401000+00:00,0 days 23:33:01.480000,65522,58620,56351,158446,84734.75
2,341257331,70299,65399,57899,40499,54206,47594,44206,0,2020-02-28 09:50:17.647000+00:00,2020-02-28 10:36:39.077000+00:00,0 days 00:46:21.430000,16093,17805,13693,40499,22022.50
3,330479809,4099,3799,2299,1099,1287,1084,0,0,2020-02-28 10:39:46.102000+00:00,2020-03-02 11:46:29.355000+00:00,3 days 01:06:43.253000,2812,2715,2299,1099,2231.25
4,319791009,70799,61399,57899,32299,56158,48876,44067,22051,2020-01-27 07:15:27.795000+00:00,2020-01-27 13:00:10.857000+00:00,0 days 05:44:43.062000,14641,12523,13832,10248,12811.00


In [ ]:
prices = prices.merge(produc)